In [1]:
import collections
import math
import os
import random
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
import pandas as pd
import re
import numpy as np
article_file = open("article1.txt", encoding='utf-8')
mydoclist = [line for line in article_file.readlines()]
v_dict_file = open("VietnameseDic.txt", encoding='utf-8')
v_dicts = [line for line in v_dict_file.readlines()]
Viet_Dictionary = []
for i in range(len(v_dicts)-1):
    k=0
    if("=====" in v_dicts[i]):
        word = v_dicts[i+1].replace("\n","")
        while("=====" not in v_dicts[i+2+k]):
            v1 = re.findall(r'[A-Z]', v_dicts[i+2+k])
            if v1:
                tag = v1[0]
            v2 = re.findall(r'^[a-z]+[a-z]+$', v_dicts[i+2+k])
            if v2:
                sense = v2[0]
            sense = v_dicts[i+2+k][9: len(v_dicts) - 1]
            row = [word, tag, sense]
            Viet_Dictionary.append(row)
            k=k+1
v_dict = pd.DataFrame(Viet_Dictionary,columns = ['Word', 'Tag', 'Sense'])

In [2]:
from collections import Counter
from sklearn import preprocessing
from pyvi.pyvi import ViTokenizer, ViPosTagger

def build_vocab_api(mydoclist):
    #Chưa loại stopword
    vocabularies = []
    types = []
    df = pd.DataFrame()
    for doc in mydoclist:
        words, word_types = ViPosTagger.postagging(ViTokenizer.tokenize(doc))
        for item in words:
            item = item.replace('_',' ')
            vocabularies.append(item)
        for word_type in word_types:
            types.append(word_type)
    df['Word'] = vocabularies
    df['Type'] = types
    return df      

def normalize(df):
    x = df.values #returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df = pd.DataFrame(x_scaled)
    return df

def tf(term, document):
    return freq(term, document)

def freq(term, document):
    return document.split().count(term)

def tf_vocab(dictionary):
    tf = Counter()
    for word in dictionary['Word']:
        tf[word] +=1 
    return tf

def build_vocab(mydoclist):
    corpus_vocab = []
    corpus_type = []
    for doc in mydoclist:
        doc.replace(".", " . ")
        words = doc.split()
        for word in words:
            corpus_vocab.append([word,'1-gram'])
            #corpus_type.append('1-gram')
        for i in range(len(words) - 3):
            if words[i][0].isupper():
                if words[i+1][0].isupper():
                    if words[i+2][0].isupper():
                        if words[i+3][0].isupper():
                            corpus_vocab.append([words[i]+' '+ words[i+1]+ ' '+ words[i+2]+ ' '+ words[i+3],'inicap4'])
                            #corpus_type.append('initcap')
                        else:
                            corpus_vocab.append([words[i]+' '+ words[i+1]+ ' '+ words[i+2],'initcap3']) 
                            #corpus_type.append('initcap'
                    else:
                        corpus_vocab.append([words[i]+' '+ words[i+1],'initcap2'])
                        #corpus_type.append('initcap')        

    for word in v_dict['Word']:
        if ' ' in word:
            for line in mydoclist:
                if (word in line.lower()) :
                    corpus_vocab.append([word,'2-ngram'])
                    #corpus_type.append('2-ngram')
                    
    for word in corpus_vocab:
        check = word[0].lower()
        for stopwords in sw_dict:
            if(check == stopwords):
                corpus_vocab.remove(word) 
    return corpus_vocab

def word_index(word, sentence):
    if ' ' in word:
        word = word.strip()[0:word.index(' ')].replace(' ', '')
    index = 0
    for i in range(len(sentence.lower().split(' ')) - 1):
        if sentence.lower().split(' ')[i] == word.lower():
            index = i       
    return index

#isupper khong kiem tra duoc tieng viet ==> dung regular expression de cai thien
# if item has index zero but is not the first word in sentence ==> remove
def sent_tokenize(sentence, v_dict_standard):
    tokenize_arr = []
    position = []
    types = []
    v_standard_dict = np.unique(v_dict_standard)
    #v_standard_dict = np.unique(v_dict)
    for i in range(len(v_standard_dict)):
        if (v_standard_dict[i][0][0] in 'ÁÂÔƠÊĐ' or v_standard_dict[i][0][0].isupper()) and len(v_standard_dict[i])>1 and v_standard_dict[i] in sentence:
            tokenize_arr.append([v_standard_dict[i],i,sentence.count(v_standard_dict[i])])
        elif(' ' in v_standard_dict[i][0] and v_standard_dict[i][0][0] in sentence.lower()):
            tokenize_arr.append([v_standard_dict[i],i,sentence.lower().count(v_standard_dict[i])])
    new_sentence = sentence
    for item in tokenize_arr:
        new_sentence = new_sentence.replace(item[0],'')
    for i in range(len(v_standard_dict)):
        if new_sentence.strip().count(v_standard_dict[i]) > 0:
            tokenize_arr.append([v_standard_dict[i],i,sentence.count(v_standard_dict[i])])
    for (word,i,k) in tokenize_arr:
        w_index= word_index(word,sentence)
        if w_index == 0 and sentence.lower().index(word.lower()) !=0:
            w_index = -1
        position.append(w_index)
        
    sentence_df = pd.DataFrame(tokenize_arr, columns = ['Word', 'Index in Vocab','Fred'])
    sentence_df['Index'] = position
    return sentence_df

def doc_tokenize(doc, vocabulary):
    doc_tokenize = []
    number_of_doc = []
    for sent in doc:
        sent_token = sent_tokenize(sent.replace('?','').replace('.','').replace('!','').replace(',',''), vocabulary)
        doc_tokenize.append([sent,len(sent),sent_token])
    return pd.DataFrame(doc_tokenize, columns = ['Sentence', 'LengthOfSentence', 'Tokenize'])

def build_lexicon(corpus):
    lexicon = set()
    for doc in corpus:
        lexicon.update([word for word in doc.split('#')])
    return lexicon

def numDocsContaining(word, doclist):
    doccount = 0
    if ' ' in word:
        word = word.strip()[0:word.index(' ')]
    for doc in doclist:
        if freq(word.lower(), doc.lower()) > 0:
            doccount +=1
    return doccount 

def l2_normalizer(vec):
    denom = np.sum([el**2 for el in vec])
    return [(el / math.sqrt(denom)) for el in vec]

def idf(word, doclist):
    n_samples = len(doclist)
    df = numDocsContaining(word, doclist)
    return np.log(n_samples / 1+df)

def extract_feature_of_token(token):
    number_of_doc = []
    frequencies = []
    tf_vocabulary = tf_vocab(v_dict)
    for word in token['Word']:
        count = numDocsContaining(word,mydoclist)
        frequency = tf_vocabulary[word]/len(np.unique(v_dict)) 
        number_of_doc.append(count)
        frequencies.append(frequency)
    token['Doc Number'] = number_of_doc
    token['Frequency'] = frequencies
    return token

In [3]:
file = open("StopWords.txt", encoding='utf-8')
lines = [line for line in file.readlines()]
sw_dict = build_lexicon(lines)
vocabulary_ = pd.DataFrame(build_vocab(mydoclist),columns = ['Word', 'Type'])

vocabulary = build_vocab_api(mydoclist)
print(vocabulary)

            Word Type
0              ﻿    N
1        Đấu giá    V
2            lần    N
3              5    M
4              ,    F
5       Agribank   Np
6          cơ mà    V
7           liệu    V
8             có    V
9            bán    V
10    thành công    N
11         Dự án    N
12        Cao ốc    V
13     văn phòng    N
14        V-Ikon   Ny
15             ?    F
16      Agribank    N
17           AMC   Ny
18           vừa    R
19            ra    V
20     thông báo    N
21           rao    V
22           bán    V
23       đấu giá    N
24       tài sản    N
25       đảm bảo    V
26            là    C
27       toàn bộ    N
28       tài sản    N
29            và    C
...          ...  ...
3276       nhiều    A
3277          bộ    N
3278         bàn    N
3279         ghế    N
3280          đá    N
3281           ,    F
3282         hai    M
3283       phòng    N
3284         học    V
3285        tiền    N
3286         chế    V
3287          và    C
3288       nhiều    A
3289      

In [4]:
document_tokenize = doc_tokenize(mydoclist, vocabulary)
data_df = extract_feature_of_token(document_tokenize['Tokenize'][1]).sort_values('Index')
print(data_df)

             Word  Index in Vocab  Fred  Index  Doc Number  Frequency
61              ở            1044     1     -1          10   0.000099
22              5              58     3     -1           2   0.000000
23              9              65     3     -1           2   0.000000
25              B              76     2     -1           0   0.000000
26              P             188     3     -1           0   0.000000
12             12              19     1     -1           0   0.000000
11              1              12     9     -1           2   0.000000
10              /              10     1     -1           0   0.000000
21              3              47     8     -1           2   0.000000
27              T             212     2     -1           0   0.000000
33            hạn             499     1     -1           1   0.000057
36              m             596     2     -1           0   0.000000
43            thô             787     1     -1           1   0.000042
52            ông   

In [5]:
feature_df = normalize(data_df[['Index in Vocab', 'Index', 'Fred', 'Frequency', 'Doc Number']])
print(feature_df)

           0         1      2         3         4
0   1.000000  0.000000  0.000  0.318182  0.344828
1   0.048263  0.000000  0.250  0.000000  0.068966
2   0.055019  0.000000  0.250  0.000000  0.068966
3   0.065637  0.000000  0.125  0.000000  0.000000
4   0.173745  0.000000  0.250  0.000000  0.000000
5   0.010618  0.000000  0.000  0.000000  0.000000
6   0.003861  0.000000  1.000  0.000000  0.068966
7   0.001931  0.000000  0.000  0.000000  0.000000
8   0.037645  0.000000  0.875  0.000000  0.068966
9   0.196911  0.000000  0.125  0.000000  0.000000
10  0.473938  0.000000  0.000  0.181818  0.034483
11  0.567568  0.000000  0.125  0.000000  0.000000
12  0.751931  0.000000  0.000  0.136364  0.034483
13  0.920849  0.000000  0.000  0.227273  0.172414
14  0.944015  0.000000  0.000  1.000000  0.206897
15  0.985521  0.000000  0.000  0.272727  0.034483
16  0.986486  0.000000  0.000  0.454545  0.310345
17  0.026062  0.000000  0.125  0.000000  0.172414
18  0.062741  0.016949  0.000  0.000000  0.241379


In [70]:
print('Data size', len(vocabulary))

# Step 2: Build the dictionary and replace rare words with UNK token.
vocabulary_size = len(np.unique(vocabulary))
print(vocabulary_size)

Data size 3306
1049


In [71]:
def build_dataset(words, n_words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [72]:
data, count, dictionary, reverse_dictionary = build_dataset(vocabulary['Word'],vocabulary_size)
#print('Most common words (+UNK)', count[:100])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

Sample data [452, 453, 57, 116, 1, 78, 454, 270, 7, 79] ['\ufeff', 'Đấu giá', 'lần', '5', ',', 'Agribank', 'cơ mà', 'liệu', 'có', 'bán']


In [73]:
data_index = 0

In [74]:
def generate_batch(data, batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    context = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window input_word skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # input word at the center of the buffer
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]  # this is the input word
            context[i * num_skips + j, 0] = buffer[target]  # these are the context words
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, context

In [75]:
data_index = 0
batch, context = generate_batch(data, batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
    print(batch[i], reverse_dictionary[batch[i]], '->', context[i, 0], reverse_dictionary[context[i, 0]])

453 Đấu giá -> 452 ﻿
453 Đấu giá -> 57 lần
57 lần -> 453 Đấu giá
57 lần -> 116 5
116 5 -> 1 ,
116 5 -> 57 lần
1 , -> 78 Agribank
1 , -> 116 5


In [76]:
batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

import tensorflow as tf
graph = tf.Graph()

In [77]:
with graph.as_default():

  # Input data.
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_context = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Look up embeddings for inputs.
    embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

  # Construct the variables for the softmax
    weights = tf.Variable(tf.truncated_normal([embedding_size, vocabulary_size],stddev=1.0 / math.sqrt(embedding_size)))
    biases = tf.Variable(tf.zeros([vocabulary_size]))
    hidden_out = tf.transpose(tf.matmul(tf.transpose(weights), tf.transpose(embed))) + biases

  # convert train_context to a one-hot format
    train_one_hot = tf.one_hot(train_context, vocabulary_size)

    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hidden_out, labels=train_one_hot))

  # Construct the SGD optimizer using a learning rate of 1.0.
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(cross_entropy)

  # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)
    
  # Add variable initializer.
    init = tf.global_variables_initializer()

In [78]:
num_steps = 100

In [86]:
def run(graph, num_steps):
    with tf.Session(graph=graph) as session:
      # We must initialize all variables before we use them.
        init.run()
        print('Initialized')

        average_loss = 0
        for step in range(num_steps):
            batch_inputs, batch_context = generate_batch(data,batch_size, num_skips, skip_window)
            feed_dict = {train_inputs: batch_inputs, train_context: batch_context}

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
            _, loss_val = session.run([optimizer, cross_entropy], feed_dict=feed_dict)
            average_loss += loss_val

            if step % 5 == 0:
                if step > 0:
                    average_loss /= 5
          # The average loss is an estimate of the loss over the last 2000 batches.
                print('Average loss at step ', step, ': ', average_loss)
                print('++++++++++++++++++++++')
                average_loss = 0

        # Note that this is expensive (~20% slowdown if computed every 500 steps)
            if step % 5 == 0:
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = reverse_dictionary[valid_examples[i]]
                    top_k = 5  # number of nearest neighbors
                    nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                    log_str = 'Nearest to %s:' % valid_word
                    print()
                    for k in range(top_k):
                        close_word = reverse_dictionary[nearest[k]]
                        log_str = '%s %s,' % (log_str, close_word)
                    print(log_str)
        final_embeddings = normalized_embeddings.eval()

In [88]:
run(graph, num_steps=1000)

Initialized
Average loss at step  0 :  7.07933807373
++++++++++++++++++++++

Nearest to cô: tâm niệm, đồng, 11,2, kỹ sư, nguyện vọng,

Nearest to học trò: Đấu giá, kỹ sư, tương đương, rất, cả,

Nearest to nhiều: thương hiệu, ghi, toàn bộ, Thế giới, chế,

Nearest to chuyển: ít, vô bờ bến, chúng tôi, Đối với, nhắc nhở,

Nearest to ngày: Hiệu trưởng, tay, 20/8, vô bờ bến, Bình,

Nearest to Lương Thế Vinh: đều, đặt, Như Cương, Sau này, xã,

Nearest to lại: thu hồi, mặc dù, đâu, Như, biến,

Nearest to ”: ấm áp, số, hướng dẫn, bắt, Bình Minh,

Nearest to để: kể, tổng, xong, 
, Như,

Nearest to đó: gia tăng, từ, 23/8, 1, Smart Factory,

Nearest to Bình Minh: TP.HCM, dẫn đầu, thị trấn, xong, cuộc,

Nearest to (: Hoàng Văn Phong, tham gia, mặt trời, chuyên ngành, mong,

Nearest to các: văn phòng, linh kiện, trên, nhau, loạt,

Nearest to ?: tổng, chi nhánh, nền, xây dựng, số ít,

Nearest to -: Mỹ, nên, cao ốc, Bao nhiêu, %,

Nearest to vào: FIR, 2010, sải, giảng dạy, gọi là,
Average loss at step

Nearest to Lương Thế Vinh: đều, đặt, Như Cương, Sau này, xã,

Nearest to lại: thu hồi, mặc dù, đâu, Như, biến,

Nearest to ”: ấm áp, số, hướng dẫn, bắt, thế hệ,

Nearest to để: kể, tổng, xong, 
, Như,

Nearest to đó: gia tăng, từ, 23/8, Smart Factory, 1,

Nearest to Bình Minh: TP.HCM, dẫn đầu, thị trấn, xong, cuộc,

Nearest to (: Hoàng Văn Phong, tham gia, mặt trời, chuyên ngành, mong,

Nearest to các: văn phòng, linh kiện, trên, nhau, loạt,

Nearest to ?: tổng, chi nhánh, nền, xây dựng, số ít,

Nearest to -: Mỹ, nên, cao ốc, Bao nhiêu, như vậy,

Nearest to vào: FIR, 2010, sải, giảng dạy, gọi là,
Average loss at step  45 :  6.68314762115
++++++++++++++++++++++

Nearest to cô: tâm niệm, đồng, 11,2, kỹ sư, nguyện vọng,

Nearest to học trò: Đấu giá, kỹ sư, tương đương, rất, cả,

Nearest to nhiều: thương hiệu, ghi, toàn bộ, Thế giới, chế,

Nearest to chuyển: ít, vô bờ bến, chúng tôi, Đối với, nhắc nhở,

Nearest to ngày: Hiệu trưởng, tay, 20/8, vô bờ bến, Bình,

Nearest to Lương Thế Vinh: đ

Nearest to các: văn phòng, linh kiện, trên, nhau, loạt,

Nearest to ?: tổng, chi nhánh, nền, xây dựng, số ít,

Nearest to -: Mỹ, nên, cao ốc, Bao nhiêu, như vậy,

Nearest to vào: FIR, 2010, giảng dạy, sải, gọi là,
Average loss at step  85 :  6.293521595
++++++++++++++++++++++

Nearest to cô: tâm niệm, đồng, 11,2, kỹ sư, nguyện vọng,

Nearest to học trò: Đấu giá, kỹ sư, tương đương, rất, cả,

Nearest to nhiều: thương hiệu, ghi, toàn bộ, Thế giới, chế,

Nearest to chuyển: ít, vô bờ bến, chúng tôi, Đối với, nhắc nhở,

Nearest to ngày: Hiệu trưởng, tay, 20/8, vô bờ bến, Bình,

Nearest to Lương Thế Vinh: đều, đặt, Như Cương, Sau này, xã,

Nearest to lại: thu hồi, mặc dù, đâu, Như, biến,

Nearest to ”: ấm áp, số, hướng dẫn, thế hệ, bắt,

Nearest to để: kể, tổng, xong, 
, Như,

Nearest to đó: gia tăng, từ, 23/8, Smart Factory, 1,

Nearest to Bình Minh: TP.HCM, dẫn đầu, thị trấn, xong, cuộc,

Nearest to (: Hoàng Văn Phong, tham gia, mặt trời, chuyên ngành, mong,

Nearest to các: văn phòng, lin

Nearest to nhiều: thương hiệu, ghi, Thế giới, chế, toàn bộ,

Nearest to chuyển: ít, vô bờ bến, chúng tôi, Đối với, nhắc nhở,

Nearest to ngày: Hiệu trưởng, tay, 20/8, vô bờ bến, bà,

Nearest to Lương Thế Vinh: đều, đặt, Như Cương, Sau này, xã,

Nearest to lại: thu hồi, mặc dù, đâu, Như, biến,

Nearest to ”: ấm áp, số, hướng dẫn, thế hệ, bắt,

Nearest to để: kể, tổng, xong, 
, Như,

Nearest to đó: gia tăng, từ, 23/8, Smart Factory, 1,

Nearest to Bình Minh: TP.HCM, thị trấn, dẫn đầu, cuộc, xong,

Nearest to (: Hoàng Văn Phong, tham gia, chuyên ngành, mặt trời, mong,

Nearest to các: văn phòng, linh kiện, trên, nhau, loạt,

Nearest to ?: tổng, chi nhánh, nền, xây dựng, số ít,

Nearest to -: Mỹ, nên, cao ốc, Bao nhiêu, như vậy,

Nearest to vào: FIR, 2010, giảng dạy, sải, gọi là,
Average loss at step  130 :  5.69119176865
++++++++++++++++++++++

Nearest to cô: tâm niệm, đồng, kỹ sư, nguyện vọng, 11,2,

Nearest to học trò: Đấu giá, kỹ sư, tương đương, rất, cả,

Nearest to nhiều: thương hiệu

Nearest to đó: gia tăng, từ, 23/8, Smart Factory, 1,

Nearest to Bình Minh: TP.HCM, thị trấn, dẫn đầu, cuộc, xong,

Nearest to (: Hoàng Văn Phong, tham gia, chuyên ngành, mặt trời, mong,

Nearest to các: văn phòng, linh kiện, trên, nhau, loạt,

Nearest to ?: tổng, chi nhánh, nền, xây dựng, số ít,

Nearest to -: Mỹ, nên, cao ốc, Bao nhiêu, như vậy,

Nearest to vào: FIR, 2010, giảng dạy, sải, gọi là,
Average loss at step  170 :  5.25321102142
++++++++++++++++++++++

Nearest to cô: tâm niệm, đồng, kỹ sư, nguyện vọng, 11,2,

Nearest to học trò: Đấu giá, kỹ sư, tương đương, rất, cả,

Nearest to nhiều: thương hiệu, ghi, chế, Thế giới, toàn bộ,

Nearest to chuyển: ít, vô bờ bến, chúng tôi, Đối với, nhắc nhở,

Nearest to ngày: Hiệu trưởng, tay, 20/8, vô bờ bến, bà,

Nearest to Lương Thế Vinh: đều, đặt, Như Cương, Sau này, xã,

Nearest to lại: thu hồi, mặc dù, đâu, Như, biến,

Nearest to ”: ấm áp, số, hướng dẫn, thế hệ, bắt,

Nearest to để: kể, tổng, xong, 
, Như,

Nearest to đó: gia tăng, từ, 

...

Nearest to ?: tổng, chi nhánh, nền, xây dựng, số ít,

Nearest to -: Mỹ, cao ốc, nên, Bao nhiêu, như vậy,

Nearest to vào: FIR, 2010, sải, giảng dạy, gọi là,
Average loss at step  780 :  3.36561808586
++++++++++++++++++++++

Nearest to cô: tâm niệm, đồng, kỹ sư, nguyện vọng, hàng,

Nearest to học trò: Đấu giá, kỹ sư, tương đương, rất, cả,

Nearest to nhiều: ghi, thương hiệu, chế, quy mô, đem,

Nearest to chuyển: ít, vô bờ bến, Đối với, chúng tôi, trai,

Nearest to ngày: Hiệu trưởng, tay, 20/8, vô bờ bến, Văn Thùy Dương,

Nearest to Lương Thế Vinh: đều, Như Cương, đặt, Sau này, đồng nghiệp,

Nearest to lại: thu hồi, Như, mặc dù, đâu, biến,

Nearest to ”: ấm áp, số, hướng dẫn, thế hệ, bắt,

Nearest to để: kể, xong, tổng, Như, 
,

Nearest to đó: gia tăng, 23/8, 1, từ, Smart Factory,

Nearest to Bình Minh: TP.HCM, thị trấn, cuộc, dẫn đầu, xong,

Nearest to (: chuyên ngành, tham gia, Hoàng Văn Phong, mặt trời, mong,

Nearest to các: văn phòng, linh kiện, trên, nhau, loạt,

Nearest to ?: tổng

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.




Nearest to ?: tổng, chi nhánh, nền, xây dựng, Đại học,

Nearest to -: Mỹ, cao ốc, nên, Bao nhiêu, mong,

Nearest to vào: FIR, 2010, sải, gọi là, giảng dạy,
Average loss at step  910 :  3.48708090782
++++++++++++++++++++++

Nearest to cô: tâm niệm, đồng, nguyện vọng, kỹ sư, bóng,

Nearest to học trò: Đấu giá, kỹ sư, tương đương, rất, cả,

Nearest to nhiều: ghi, thương hiệu, quy mô, chế, đem,

Nearest to chuyển: ít, vô bờ bến, Đối với, trai, chúng tôi,

Nearest to ngày: Hiệu trưởng, tay, 20/8, vô bờ bến, Văn Thùy Dương,

Nearest to Lương Thế Vinh: đều, Như Cương, đặt, Sau này, đồng nghiệp,

Nearest to lại: thu hồi, Như, mặc dù, đâu, biến,

Nearest to ”: ấm áp, số, hướng dẫn, thế hệ, bắt,

Nearest to để: kể, xong, tổng, Như, 
,

Nearest to đó: gia tăng, 23/8, 1, Smart Factory, từ,

Nearest to Bình Minh: TP.HCM, thị trấn, cuộc, dẫn đầu, xong,

Nearest to (: chuyên ngành, tham gia, Hoàng Văn Phong, mong, mặt trời,

Nearest to các: văn phòng, linh kiện, trên, nhau, loạt,

Nearest to ?: tổn

Nearest to nhiều: ghi, thương hiệu, chế, quy mô, đem,

Nearest to chuyển: ít, vô bờ bến, Đối với, trai, nhắc nhở,

Nearest to ngày: Hiệu trưởng, tay, 20/8, vô bờ bến, 
,

Nearest to Lương Thế Vinh: đều, Như Cương, đặt, Sau này, đồng nghiệp,

Nearest to lại: thu hồi, Như, mặc dù, đâu, biến,

Nearest to ”: ấm áp, số, hướng dẫn, Nghề, thế hệ,

Nearest to để: kể, xong, Như, tổng, 
,

Nearest to đó: gia tăng, 23/8, 1, Smart Factory, từ,

Nearest to Bình Minh: TP.HCM, thị trấn, cuộc, xong, dẫn đầu,

Nearest to (: chuyên ngành, tham gia, Hoàng Văn Phong, mong, mặt trời,

Nearest to các: văn phòng, linh kiện, trên, nhau, loạt,

Nearest to ?: chi nhánh, tổng, nền, xây dựng, Đại học,

Nearest to -: Mỹ, cao ốc, nên, Bao nhiêu, mong,

Nearest to vào: FIR, 2010, sải, gọi là, giảng dạy,
Average loss at step  955 :  3.31925601959
++++++++++++++++++++++

Nearest to cô: tâm niệm, đồng, nguyện vọng, kỹ sư, hàng,

Nearest to học trò: Đấu giá, kỹ sư, tương đương, rất, cả,

Nearest to nhiều: ghi, thương hi


Nearest to Bình Minh: TP.HCM, thị trấn, cuộc, dẫn đầu, xong,

Nearest to (: chuyên ngành, tham gia, Hoàng Văn Phong, mong, mặt trời,

Nearest to các: văn phòng, linh kiện, trên, nhau, loạt,

Nearest to ?: chi nhánh, tổng, nền, xây dựng, Nguyễn Thị Măng,

Nearest to -: Mỹ, cao ốc, nên, Bao nhiêu, mong,

Nearest to vào: FIR, 2010, sải, gọi là, giảng dạy,
Average loss at step  995 :  3.15451879501
++++++++++++++++++++++

Nearest to cô: tâm niệm, đồng, nguyện vọng, kỹ sư, bóng,

Nearest to học trò: Đấu giá, kỹ sư, tương đương, rất, cả,

Nearest to nhiều: ghi, thương hiệu, quy mô, chế, đem,

Nearest to chuyển: ít, vô bờ bến, Đối với, trai, nhắc nhở,

Nearest to ngày: Hiệu trưởng, tay, 20/8, vô bờ bến, 
,

Nearest to Lương Thế Vinh: đều, Như Cương, đặt, Sau này, đồng nghiệp,

Nearest to lại: thu hồi, Như, mặc dù, đâu, biến,

KeyError: 2000